In [2]:
import pandas as pd
import numpy as np
import torch
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
class Activate(torch.nn.Module):
    def forward(self, x):
        a = torch.exp(x[:, 0]).unsqueeze(1)
        b = torch.nn.functional.softplus(x[:, 1]).unsqueeze(1)
        
        res = torch.cat([a,b], dim=1)
        
        return res

In [137]:
# Here we define our model as a class
class LSTM(torch.nn.Module):

    def __init__(self, input_dim, hidden_dim, batch_size, output_dim=2, num_layers=2, device='cpu'):
        super(LSTM, self).__init__()
        self.device = device
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.batch_size = batch_size
        self.num_layers = num_layers

        self.lstm = torch.nn.LSTM(self.input_dim, self.hidden_dim, self.num_layers, batch_first=True)

        self.linear = torch.nn.Linear(self.hidden_dim, output_dim)


    def forward(self, input):
        batch_size = input.shape[0]
        
        h0 = torch.zeros(self.num_layers, batch_size, self.hidden_dim).to(self.device)
        c0 = torch.zeros(self.num_layers, batch_size, self.hidden_dim).to(self.device)

        out = input.view(batch_size, -1, self.input_dim)
        out, _ = self.lstm(out, (h0, c0))  
        
        out = out[:, -1, :]
        
        out = self.linear(out)
        
        a = torch.exp(out[:, 0]).unsqueeze(1)
        b = torch.nn.functional.softplus(out[:, 1]).unsqueeze(1)
        
        out = torch.cat([a,b], dim=1)
        
        
#         lstm_out, self.hidden = self.lstm(input.view(batch_size, -1, 1))
#         y_pred = self.linear(lstm_out[-1].contiguous().view(batch_size, -1))
        return out

model = LSTM(1, 32, batch_size=batch_size, output_dim=2, num_layers=2, device=device).to(device)
optim = torch.optim.Adam(model.parameters(), lr=0.004)

In [140]:
x = x_lstm

In [141]:
from torch.utils.data import TensorDataset, DataLoader

x = torch.FloatTensor(x).to(device)
y = torch.FloatTensor(y).to(device)

dataset = TensorDataset(x, y)
dataloader = DataLoader(dataset, batch_size=batch_size, )


# my_dataset = utils.TensorDataset(x, y) # create your datset
# my_dataloader = utils.DataLoader(my_dataset, batch_size=batch_size)

In [142]:
def weibull_loss(y_true, ab):
    y_ = y_true[:, 0]
    
    a_ = ab[:, 0]
    b_ = ab[:, 1]
    
    haz0 = ((y_ + 1e-35) / a_).pow(b_)
    haz1 = ((y_ + 1) / a_).pow(b_)
    
    log_val = torch.log(torch.exp(haz1 - haz0) - 1.0)
    
    return -1 * (log_val - haz1).mean()

def weibull_mean(a, b):
    # Continuous mean. Theoretically at most 1 step below discretized mean 
    # E[T ] <= E[Td] + 1 true for positive distributions. 
    from scipy.special import gamma
    return a*gamma(1.0+1.0/b)

def weibull_mode(a, b):
    # Continuous mode. 
    # TODO (mathematically) prove how close it is to discretized mode
    mode = a*np.power((b-1.0)/b,1.0/b)
    mode[b<=1.0]=0.0
    return mode

In [149]:
def train_epoch(model, optim, dataloader, device):
    loss_ar = []
    for data_x, data_y in dataloader:
        res = model(data_x)
        loss = weibull_loss(data_y, res)
        loss_ar.append(loss.item())

        optim.zero_grad()
        loss.backward()
#         torch.nn.utils.clip_grad_norm_(model.parameters(), 1)
        optim.step()
    return np.mean(loss_ar)

def test(model, dataloader, device):
    pred = []
    with torch.no_grad():
        for data, _ in dataloader:
            pred.extend(model(data).cpu().detach().numpy())
    
#     nums = [weibull_mean(x[0], x[1]) for x in pred]
    nums = weibull_mode(pred[:,0], pred[:,1])
    return sMAPE(y.squeeze(1).cpu().detach().numpy(), nums)

In [ ]:
for epoch in range(100):
#     loss = train_epoch(model, optim, dataloader, device)
    smape = test(model, dataloader, device)
    print(epoch, loss, smape)

In [376]:
pred = None
with torch.no_grad():
    pred = model(x).cpu().detach().numpy()

In [378]:
nums = [weibull_mean(x[0], x[1]) for x in pred]

In [379]:
nums[:10], y[:10]

([8.050626543655833,
  7.7008617161123505,
  7.6089758059659545,
  6.672095845386341,
  6.5842507629710445,
  6.566309337091586,
  7.297705630494468,
  7.7684387061981255,
  8.873279097635093,
  9.078957406321004],
 tensor([[5.],
         [5.],
         [7.],
         [7.],
         [7.],
         [7.],
         [7.],
         [6.],
         [6.],
         [5.]]))

In [487]:
pred = model(x).cpu().detach().numpy()

In [488]:
nums = weibull_mode(pred[:, 0], pred[:, 1])

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:4: RuntimeWarning: invalid value encountered in power
  after removing the cwd from sys.path.


In [491]:
nums.mean()

9.692931

In [490]:
 y.detach().numpy().mean()

11.6245165

In [489]:
nums[:10], y.detach().numpy()[:10]

(array([6.0077686, 5.7603803, 5.843462 , 5.393385 , 5.3903713, 5.3190107,
        5.6041636, 5.95658  , 6.55284  , 6.952653 ], dtype=float32),
 array([[5.],
        [5.],
        [7.],
        [7.],
        [7.],
        [7.],
        [7.],
        [6.],
        [6.],
        [5.]], dtype=float32))

In [383]:
y.squeeze(1).cpu().detach().numpy()

array([5., 5., 7., ..., 2., 1., 3.], dtype=float32)

In [385]:
sMAPE(y.squeeze(1).cpu().detach().numpy(), nums)

0.573123821491823

In [ ]:
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes, batch_size=None, device='cpu', pv_num=None):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size
        self.input_size = input_size
        self.num_layers = num_layers
        self.batch_size = batch_size
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, bidirectional=True)
        self.fc1 = nn.Linear(hidden_size*2*sequence_length, 32)
        self.fc2 = nn.Linear(32, num_classes)

    def forward(self, x):
        batch_size = x.size(0)
        out = x.view(batch_size, 1, x.size(1), x.size(2))

        h0 = torch.zeros(2, batch_size, self.hidden_size)
        c0 = torch.zeros(2, batch_size, self.hidden_size)
        
        h0 = h0.to(device)
        c0 = c0.to(device)
        
        out = out.view(batch_size, -1, self.input_size)
        out, _ = self.lstm(out, (h0, c0))  

        out = out.contiguous().view(batch_size, -1)

        out = torch.relu(self.fc1(out))
        out = self.fc2(out)
        out = torch.relu(out)
        out = torch.softmax(out, 1)

        return out